In [1]:
import numpy as np
import json
import cvxpy as cp
from scipy import sparse as sp
import scipy.special as sps
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
import math

from src.problems.utils import sample_data_for_group
from src.problems.problems import compute_producer_optimal_solution, _compute_consumer_optimal_solution_cvar_relaxed_base, _compute_consumer_optimal_solution_cvar
from src.problems.gradient_problem import compute_consumer_optimal_solution_cvar_grad, compute_consumer_optimal_with_augmented_lagrangian_cvar

In [27]:
def _compute_consumer_optimal_solution_cvar_relaxed_topk_rounding(
    allocations: np.ndarray,
    k_rec: int,
) -> tuple[float, np.ndarray]:
    idxs = allocations.argsort(axis=1)[:, -k_rec:]
    alls = np.zeros_like(allocations)
    alls[np.arange(allocations.shape[0])[:, None], idxs] = 1

    return alls

def _compute_consumer_optimal_solution_cvar_relaxed_naive_rounding(
    allocations: np.ndarray,
) -> tuple[float, np.ndarray]:
    alls = allocations.round().astype(int)

    return alls

In [4]:
# load data
with open("../../data/amazon_predictions.npy", "rb") as f:
    REL_MATRIX = np.load(f)

with open("../../data/amazon_user_groups.json", "r") as f:
    GROUPS_MAP = json.load(f)

In [5]:
N_CONSUMERS = 300
N_PRODUCERS = 300
ALPHA = 0.5
GAMMA = 0.5
GROUP_KEY = "usage_group"
K_REC = 10
SOLVER = cp.GUROBI

In [6]:
def compute_perc_prod_below_threshold(
    allocation: np.ndarray,
    threshold: float,
) -> float:
    producer_allocations = allocation.sum(axis=0)
    num_of_producers_below_threshold = np.sum(producer_allocations < threshold)
    return num_of_producers_below_threshold / len(producer_allocations)

def compute_mean_prod_below_threshold(
    allocation: np.ndarray,
    threshold: float,
) -> float:
    producer_allocations = allocation.sum(axis=0)
    return np.mean(math.ceil(threshold) - producer_allocations[producer_allocations < threshold])

def compute_mean_consumer_utility(
    allocation: np.ndarray,
    rel_matrix: np.ndarray,
) -> float:
    return (allocation * rel_matrix).sum() / allocation.sum()

def compute_perc_cons_above_threshold(
    allocation: np.ndarray,
    k_rec: int
) -> float:
    consumer_allocations = allocation.sum(axis=1)
    num_of_consumers_above_threshold = np.sum(consumer_allocations > k_rec)
    return num_of_consumers_above_threshold / len(consumer_allocations)

def compute_perc_cons_below_threshold(
    allocation: np.ndarray,
    k_rec: int
) -> float:
    consumer_allocations = allocation.sum(axis=1)
    num_of_consumers_below_threshold = np.sum(consumer_allocations < k_rec)
    return num_of_consumers_below_threshold / len(consumer_allocations)

def compute_mean_cons_above_threshold(
    allocation: np.ndarray,
    k_rec: int
) -> float:
    consumer_allocations = allocation.sum(axis=1)
    return np.mean(k_rec - consumer_allocations[consumer_allocations > k_rec])


def compute_mean_cons_below_threshold(
    allocation: np.ndarray,
    k_rec: int
) -> float:
    consumer_allocations = allocation.sum(axis=1)
    return np.mean(k_rec - consumer_allocations[consumer_allocations < k_rec])

def compute_std_between_groups(allocations: np.ndarray, group_assignments: np.ndarray) -> float:
    unique_groups, group_indices = np.unique(group_assignments, return_inverse=True)
    num_groups = len(unique_groups)
    group_masks = [group_indices == i for i in range(num_groups)]
    group_sizes = np.array([mask.sum() for mask in group_masks])

    means = []
    for mask, size in zip(group_masks, group_sizes):
        group_alloc = allocations[mask]
        mean = np.mean(group_alloc.sum(axis=0))
        means.append(mean)

    return np.std(means)

In [29]:
results = []

for size in [100]:
    print(f"Running for size {size}")
    for k_rec in [5, 10, 25]:
        print(f"Running for k_rec {k_rec}")
        for run in range(3):
            rel_matrix_sampled, consumer_ids, group_assignments = sample_data_for_group(
                n_consumers=size,
                n_producers=size,
                groups_map=GROUPS_MAP,
                group_key=GROUP_KEY,
                data=REL_MATRIX,
                naive_sampling=True,
                seed=run,
            )
            producer_max_min_utility, _ = compute_producer_optimal_solution(
                rel_matrix=rel_matrix_sampled,
                k_rec=k_rec,
                solver=SOLVER,
            )
            start_time = time()
            _, optimal_allocation = _compute_consumer_optimal_solution_cvar(
                rel_matrix=rel_matrix_sampled,
                k_rec=k_rec,
                producer_max_min_utility=producer_max_min_utility,
                gamma=GAMMA,
                group_assignments=group_assignments,
                alpha=ALPHA,
                solver=SOLVER,
            )
            optimal_time = time() - start_time

            start_time = time()
            _, _base_cvar_allocation = _compute_consumer_optimal_solution_cvar_relaxed_base(
                rel_matrix=rel_matrix_sampled,
                k_rec=k_rec,
                producer_max_min_utility=producer_max_min_utility,
                gamma=GAMMA,
                group_assignments=group_assignments,
                alpha=ALPHA,
                solver=SOLVER,
            )
            relaxed_time = time() - start_time

            rounded_cvar_allocation = _compute_consumer_optimal_solution_cvar_relaxed_naive_rounding(
                allocations=_base_cvar_allocation,
            )
            topk_cvar_allocation = _compute_consumer_optimal_solution_cvar_relaxed_topk_rounding(
                allocations=_base_cvar_allocation,
                k_rec=k_rec,
            )

            start_time = time()
            _base_cvar_grad_allocations, _ = compute_consumer_optimal_solution_cvar_grad(
                rel_matrix=rel_matrix_sampled,
                k_rec=k_rec,
                producer_max_min_utility=producer_max_min_utility,
                gamma=GAMMA,
                group_assignments=group_assignments,
                alpha=ALPHA,
                hidden_dim=size * 2,
                max_epochs=30000,
                verbose=False,
                max_patience=30,
                patience_delta=1e-2
            )
            grad_time = time() - start_time

            rounded_cvar_grad_allocation = _compute_consumer_optimal_solution_cvar_relaxed_naive_rounding(
                allocations=_base_cvar_grad_allocations,
            )
            topk_cvar_grad_allocation = _compute_consumer_optimal_solution_cvar_relaxed_topk_rounding(
                allocations=_base_cvar_grad_allocations,
                k_rec=k_rec,
            )

            start_time = time()
            _base_cvar_lang_allocation  = compute_consumer_optimal_with_augmented_lagrangian_cvar(
                k_rec=k_rec,
                rel_matrix=rel_matrix_sampled,
                producer_max_min_utility=producer_max_min_utility,
                gamma=GAMMA,
                group_assignments=group_assignments,
                alpha_cvar=ALPHA,
                verbose=False,
                max_epochs=40000,
            )
            lang_time = time() - start_time
            rounded_cvar_lang_allocation = _compute_consumer_optimal_solution_cvar_relaxed_naive_rounding(
                allocations=_base_cvar_lang_allocation,
            )
            print(rounded_cvar_lang_allocation)
            topk_cvar_lang_allocation = _compute_consumer_optimal_solution_cvar_relaxed_topk_rounding(
                allocations=_base_cvar_lang_allocation,
                k_rec=k_rec,
            )


            stats = {}
            for allocation, method in zip(
                [optimal_allocation, rounded_cvar_allocation, topk_cvar_allocation, rounded_cvar_grad_allocation, topk_cvar_grad_allocation, rounded_cvar_lang_allocation, topk_cvar_lang_allocation],
                ["Baseline", "LP Round", "LP TopK", "Grad Round", "Grad TopK", "Lang Round", "Lang TopK"]
            ):
                stats["$\mathbb{E}[{U_{C}}]$_" + method] = compute_mean_consumer_utility(
                    allocation=allocation,
                    rel_matrix=rel_matrix_sampled,
                )
                stats["$\sigma(\mathbb{E}[U_{G}])$_" + method] = compute_std_between_groups(
                    allocations=allocation,
                    group_assignments=group_assignments,
                )
                stats["$\#(U_{P} < U_{P\min})/N_{P}$_" + method] = compute_perc_prod_below_threshold(
                    allocation=allocation,
                    threshold=producer_max_min_utility * GAMMA,
                )
                stats["$\sum(V - U_{P})/N_{P^{b}}$_" + method] = compute_mean_prod_below_threshold(
                    allocation=allocation,
                    threshold=producer_max_min_utility * GAMMA,
                )
                stats["$\#(A_{C} > k)/N_{C}$_" + method] = compute_perc_cons_above_threshold(
                    allocation=allocation,
                    k_rec=k_rec,
                )
                stats["$\sum(A_{C} - k)/N_{A_{c}^{a}}$_" + method] = compute_mean_cons_above_threshold(
                    allocation=allocation,
                    k_rec=k_rec,
                )
                stats["$\#(A_{C} < k)/N_{C}$_" + method] = compute_perc_cons_below_threshold(
                    allocation=allocation,
                    k_rec=k_rec,
                )
                stats["$\sum(k - A_{C})/N_{A_{c}^{b}}$_" + method] = compute_mean_cons_below_threshold(
                    allocation=allocation,
                    k_rec=k_rec,
                )

            results.append({
                "$N_{c}$, N_{p}$": size,
                "k": k_rec,
                "optimal_time": optimal_time,
                "relaxed_time": relaxed_time,
                "grad_time": grad_time,
                **stats,
                })

<>:105: SyntaxWarning: invalid escape sequence '\m'
<>:109: SyntaxWarning: invalid escape sequence '\s'
<>:113: SyntaxWarning: invalid escape sequence '\#'
<>:117: SyntaxWarning: invalid escape sequence '\s'
<>:121: SyntaxWarning: invalid escape sequence '\#'
<>:125: SyntaxWarning: invalid escape sequence '\s'
<>:129: SyntaxWarning: invalid escape sequence '\#'
<>:133: SyntaxWarning: invalid escape sequence '\s'
<>:105: SyntaxWarning: invalid escape sequence '\m'
<>:109: SyntaxWarning: invalid escape sequence '\s'
<>:113: SyntaxWarning: invalid escape sequence '\#'
<>:117: SyntaxWarning: invalid escape sequence '\s'
<>:121: SyntaxWarning: invalid escape sequence '\#'
<>:125: SyntaxWarning: invalid escape sequence '\s'
<>:129: SyntaxWarning: invalid escape sequence '\#'
<>:133: SyntaxWarning: invalid escape sequence '\s'
/var/folders/9l/_y964wyn5fx6bcqmbqjpd9780000gp/T/ipykernel_38848/669903939.py:105: SyntaxWarning: invalid escape sequence '\m'
  stats["$\mathbb{E}[{U_{C}}]$_" + method

Running for size 100
Running for k_rec 5
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


/Users/dominykas.seputis/github/msc-thesis/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dominykas.seputis/github/msc-thesis/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/dominykas.seputis/github/msc-thesis/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]
[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Running for k_rec 10
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]
[[0 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Running for k_rec 25
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 1]]
[[0 1 1 ... 0 0 1]
 [1 0 1 ... 0 0 0]
 [0 0 1 ... 1 0 1]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 1 ... 0 1 1]

In [30]:
df = pd.DataFrame(results)

In [9]:
df = pd.read_csv("experiments.csv")

In [31]:
df.to_csv("output.csv", index=False)

In [32]:
df.columns

Index(['$N_{c}$, N_{p}$', 'k', 'optimal_time', 'relaxed_time', 'grad_time',
       '$\mathbb{E}[{U_{C}}]$_Baseline',
       '$\sigma(\mathbb{E}[U_{G}])$_Baseline',
       '$\#(U_{P} < U_{P\min})/N_{P}$_Baseline',
       '$\sum(V - U_{P})/N_{P^{b}}$_Baseline',
       '$\#(A_{C} > k)/N_{C}$_Baseline',
       '$\sum(A_{C} - k)/N_{A_{c}^{a}}$_Baseline',
       '$\#(A_{C} < k)/N_{C}$_Baseline',
       '$\sum(k - A_{C})/N_{A_{c}^{b}}$_Baseline',
       '$\mathbb{E}[{U_{C}}]$_LP Round',
       '$\sigma(\mathbb{E}[U_{G}])$_LP Round',
       '$\#(U_{P} < U_{P\min})/N_{P}$_LP Round',
       '$\sum(V - U_{P})/N_{P^{b}}$_LP Round',
       '$\#(A_{C} > k)/N_{C}$_LP Round',
       '$\sum(A_{C} - k)/N_{A_{c}^{a}}$_LP Round',
       '$\#(A_{C} < k)/N_{C}$_LP Round',
       '$\sum(k - A_{C})/N_{A_{c}^{b}}$_LP Round',
       '$\mathbb{E}[{U_{C}}]$_LP TopK', '$\sigma(\mathbb{E}[U_{G}])$_LP TopK',
       '$\#(U_{P} < U_{P\min})/N_{P}$_LP TopK',
       '$\sum(V - U_{P})/N_{P^{b}}$_LP TopK', '$\#(A_{C} > k)

In [33]:
GROUP_BY_COLS = ["$N_{c}$, N_{p}$", "k"]
METHODS = ["Baseline", "LP Round", "LP TopK", "Grad Round", "Grad TopK", "Lang Round", "Lang TopK"]
BASE_METRICS = [
    "$\mathbb{E}[{U_{C}}]$",
    "$\sigma(\mathbb{E}[U_{G}])$",
    "$\#(U_{P} < U_{P\min})/N_{P}$",
    "$\sum(V - U_{P})/N_{P^{b}}$",
    "$\#(A_{C} > k)/N_{C}$",
    "$\sum(A_{C} - k)/N_{A_{c}^{a}}$",
    "$\#(A_{C} < k)/N_{C}$",
    "$\sum(k - A_{C})/N_{A_{c}^{b}}$",
]

def aggregate_wide(df: pd.DataFrame):
    # build the dict of aggregations
    agg_dict = {
        f"{base}_{m}": ["mean", "std"]
        for m in METHODS
        for base in BASE_METRICS
    }
    # group and flatten the MultiIndex
    grouped = df.groupby(GROUP_BY_COLS).agg(agg_dict)
    # flatten columns: e.g. ("mean_consumer_utility_optimal","mean") → "mean_consumer_utility_optimal_mean"
    grouped.columns = [
        f"{col[0]}_{col[1]}" for col in grouped.columns
    ]
    return grouped.reset_index()

agg_df = aggregate_wide(df)

def aggregate_long(df: pd.DataFrame):
    # melt into a tall table
    value_vars = [
        f"{base}_{m}"
        for m in METHODS
        for base in BASE_METRICS
    ]
    df_long = df.melt(
        id_vars=GROUP_BY_COLS,
        value_vars=value_vars,
        var_name="metric_method",
        value_name="value",
    )
    method_pattern = "|".join(re.escape(m) for m in METHODS)
    extract_re = rf"(?P<metric>.*)_(?P<Method>{method_pattern})$"

    # do the extract:
    df_long[["metric", "Method"]] = df_long["metric_method"].str.extract(extract_re)
    # split “mean_consumer_utility_optimal” → (“mean_consumer_utility”, “optimal”)
    # now group
    return (
        df_long
        .groupby(GROUP_BY_COLS + ["Method", "metric"])["value"]
        .agg(mean="mean", std="std")
        .reset_index()
    )

agg_long_df = aggregate_long(df)

<>:4: SyntaxWarning: invalid escape sequence '\m'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\#'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\#'
<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\#'
<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\m'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\#'
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\#'
<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\#'
<>:11: SyntaxWarning: invalid escape sequence '\s'
/var/folders/9l/_y964wyn5fx6bcqmbqjpd9780000gp/T/ipykernel_38848/317195415.py:4: SyntaxWarning: invalid escape sequence '\m'
  "$\mathbb{E}[{U_{C}}]$",
/var/folders/9l/_y964wyn5fx6bcqmbqjpd9780000

In [34]:
# 1) combine mean+std into one column
def fmt(x, y):
    if pd.isna(x):
        return ""
    return f"{x:.2f} ({y:.2f})"

agg_long_df["mean_std"] = agg_long_df.apply(
    lambda row: fmt(row["mean"], row["std"]),
    axis=1
)

# 2) pivot so metrics become columns
pivot = (
    agg_long_df
    .pivot_table(
        index=["$N_{c}$, N_{p}$", "k", "Method"],
        columns="metric",
        values="mean_std",
        aggfunc="first"
    )
    # flatten the column Index
    .rename_axis(None, axis=1)
    .reset_index()
)

# 3) export to LaTeX
latex = pivot.to_latex(
    index=False,
    na_rep="",
    column_format="lll" + "r" * (pivot.shape[1] - 3),
    longtable=False,
    caption="Performance by method and metric",
    label="tab:results",
    escape=False,
    bold_rows=False,
    multicolumn=True,
)

# add \resizebox{\textwidth}{!}{\begin{tabular}
latex = latex.replace(
    "\\begin{tabular}",
    "\\resizebox{\\textwidth}{!}{\\begin{tabular}"
).replace(
    "\\end{tabular}",
    "\\end{tabular}}"
)

print(latex)

\begin{table}
\caption{Performance by method and metric}
\label{tab:results}
\resizebox{\textwidth}{!}{\begin{tabular}{lllrrrrrrrr}
\toprule
$N_{c}$, N_{p}$ & k & Method & $\#(A_{C} < k)/N_{C}$ & $\#(A_{C} > k)/N_{C}$ & $\#(U_{P} < U_{P\min})/N_{P}$ & $\mathbb{E}[{U_{C}}]$ & $\sigma(\mathbb{E}[U_{G}])$ & $\sum(A_{C} - k)/N_{A_{c}^{a}}$ & $\sum(V - U_{P})/N_{P^{b}}$ & $\sum(k - A_{C})/N_{A_{c}^{b}}$ \\
\midrule
100 & 5 & Baseline & 0.00 (0.00) & 0.00 (0.00) & 0.00 (0.00) & 0.94 (0.00) & 1.19 (0.00) &  &  &  \\
100 & 5 & Grad Round & 0.00 (0.00) & 0.03 (0.01) & 0.02 (0.01) & 0.92 (0.00) & 1.19 (0.00) & -1.00 (0.00) & 1.00 (0.00) &  \\
100 & 5 & Grad TopK & 0.00 (0.00) & 0.00 (0.00) & 0.03 (0.01) & 0.92 (0.00) & 1.19 (0.00) &  & 1.00 (0.00) &  \\
100 & 5 & LP Round & 0.01 (0.01) & 0.00 (0.00) & 0.00 (0.00) & 0.94 (0.00) & 1.19 (0.00) &  &  & 1.00 (0.00) \\
100 & 5 & LP TopK & 0.00 (0.00) & 0.00 (0.00) & 0.00 (0.00) & 0.94 (0.00) & 1.19 (0.00) &  &  &  \\
100 & 5 & Lang Round & 0.00 (0.00)